In [3]:
# earth engine
import ee
import geemap.eefolium as geemap
#from geemap import geojson_to_ee, ee_to_geojson

# analysis and mapping
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
#import json
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from pprint import pprint

# file management stuff
import datetime
import glob
import urllib
import zipfile
from zipfile import ZipFile 

In [4]:
ee.Authenticate() 

Enter verification code: 4/1AX4XfWhkyiyVsIeGpwM6Sh-ehmwWmZn4R2gLN3VADh5-agU7tivy8CqSKJY

Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Add EE drawing method to folium.
#folium.Map.add_ee_layer = add_ee_layer
folium.plugins.DualMap.add_ee_layer = add_ee_layer

In [7]:
# Function to sample values by HOLC zones
def sample_holc_layer(zones, dataset, scale):
  resultList = []
  for feat in zones.getInfo()['features']:
    name = feat['properties']['level_0']
    print(name)
    region = ee.Feature(zones.filter(ee.Filter.eq('level_0', name)).first())
    # Reduce the region. The region parameter is the Feature geometry.
    meanDictionary = dataset.reduceRegion(**{'reducer': ee.Reducer.mean(),'geometry': region.geometry(),'scale': scale,'maxPixels': 1e9})
    # The result is a Dictionary.  Print it.
    #resultDict[name]=meanDictionary.getInfo()
    resultList.append(meanDictionary.getInfo())
    pprint(meanDictionary.getInfo())

  datadf = pd.DataFrame(resultList)
  datadf.reset_index(inplace=True)
  datadf['joinid']=datadf['index'].apply(str)

  return datadf

In [8]:
url = r'https://dsl.richmond.edu/panorama/redlining/static/fullshpfile.zip'
dwnfile = url.split('/')[-1]

# Copy a network object to a local file
urllib.request.urlretrieve(url, dwnfile)

# specifying the zip file name 
file_name = dwnfile
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zf: 
    # printing all the contents of the zip file 
    zf.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zf.extractall() 
    print('Done!')

del zf

File Name                                             Modified             Size
fullshpfile/city_population_data.csv           2020-08-03 09:03:56        37437
fullshpfile/HOLC_Cities.gdb/                   2020-10-07 13:29:52            0
fullshpfile/HOLC_Cities.gdb/a00000001.freelist 2020-10-07 13:29:26         4440
fullshpfile/HOLC_Cities.gdb/a00000001.gdbindexes 2020-06-11 12:28:36          110
fullshpfile/HOLC_Cities.gdb/a00000001.gdbtable 2020-10-07 13:29:26          353
fullshpfile/HOLC_Cities.gdb/a00000001.gdbtablx 2020-10-07 13:29:26         5152
fullshpfile/HOLC_Cities.gdb/a00000001.TablesByName.atx 2020-10-07 13:29:26         4118
fullshpfile/HOLC_Cities.gdb/a00000002.gdbtable 2020-06-11 12:28:36         2055
fullshpfile/HOLC_Cities.gdb/a00000002.gdbtablx 2020-06-11 12:28:36         5152
fullshpfile/HOLC_Cities.gdb/a00000003.gdbindexes 2020-06-11 12:28:36           42
fullshpfile/HOLC_Cities.gdb/a00000003.gdbtable 2020-06-11 12:28:36         1313
fullshpfile/HOLC_Cities.gdb/

In [10]:
!pip install google.colab

     |████████████████████████████████| 72 kB 1.5 MB/s eta 0:00:011
     |████████████████████████████████| 64 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 758 kB 19.4 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 27.4 MB/s eta 0:00:01
     |████████████████████████████████| 11.8 MB 21.5 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 484 kB 22.3 MB/s eta 0:00:01
     |████████████████████████████████| 245 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 118 kB 20.7 MB/s eta 0:00:01


  Created wheel for google.colab: filename=google_colab-1.0.0-py2.py3-none-any.whl size=102290 sha256=e265aafe16640dc8dcd224a98dddfe64fb1641ee3baf0adf13fbcbebd47f7674
  Stored in directory: /Users/dongmeichen/Library/Caches/pip/wheels/e4/e7/91/b2736701bca00e273fdc79f80e2727b558e0903d81b758eb69
  ERROR: Command errored out with exit status 1:
   command: /Users/dongmeichen/opt/anaconda3/envs/geoenv/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/h2/gd54d6856bbbw2tmjwhxkj6c0000gn/T/pip-install-s92aw75s/pandas/setup.py'"'"'; __file__='"'"'/private/var/folders/h2/gd54d6856bbbw2tmjwhxkj6c0000gn/T/pip-install-s92aw75s/pandas/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/h2/gd54d6856bbbw2tmjwhxkj6c0000gn/T/pip-wheel-2_w0d7el
       cwd: /private/var/folders/h2/gd54d6856bbbw2tmjwhxkj6c000

  copying pandas/tests/test_config.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/tests
  copying pandas/tests/test_window.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/tests
  copying pandas/tests/test_strings.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/tests
  copying pandas/tests/test_base.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/tests
  creating build/lib.macosx-10.9-x86_64-3.8/pandas/_libs
  copying pandas/_libs/__init__.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/_libs
  creating build/lib.macosx-10.9-x86_64-3.8/pandas/plotting
  copying pandas/plotting/_converter.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/plotting
  copying pandas/plotting/_tools.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/plotting
  copying pandas/plotting/_style.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/plotting
  copying pandas/plotting/__init__.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/plotting
  copying pandas/plotting/_core.py -> build/lib.macosx-10.9-x86_64-3.8/pandas/plotting
  copyi

  Created wheel for portpicker: filename=portpicker-1.2.0-py3-none-any.whl size=13371 sha256=e18e7735ba7a26990b4224c060aec5fc622b6176b7e62d8b9748c2ce726618ee
  Stored in directory: /Users/dongmeichen/Library/Caches/pip/wheels/b9/10/77/cad2fb4f8ab57cc7cecb92b9b1b0548ac932b8f81774003683
  Created wheel for tornado: filename=tornado-4.5.3-cp38-cp38-macosx_10_9_x86_64.whl size=420780 sha256=62bc5790bee9cbe56136d85c0607dfa46860aa6db0624b2b32d078dc1cfac016
  Stored in directory: /Users/dongmeichen/Library/Caches/pip/wheels/20/fe/26/0c03adccd0cec2744261ec6b8380fbe5e726d1ea4cf3a9370e
Successfully built google.colab portpicker tornado
Failed to build pandas
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.32.0
    Uninstalling google-auth-1.32.0:
      Successfully uninstalled google-auth-1.32.0
  Attempting uninstall: p

In [9]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'